# 04_Data_Cleaning_and_EDA
This notebook explores the Exploratory Data Analysis (EDA) of both Numerical and Categorical variables, to produce a final, clean dataset ready for AI pipeline.

## Data Understanding:
* Analyzing distribution of drugs, conditions, and ratings.
* **Data Quality:** Identifying and removing noise (short reviews).
* **Preprocessing:** preparing a "Cleaned" table for the AI pipeline.

#### 1. LOAD DATA

In [0]:
from pyspark.sql.functions import length, col, max, min, count, when, avg, stddev
from pyspark.sql.functions import col, count, when, isnan

In [0]:
# Setup Context
catalog = "safety_signal_catalog"
schema  = "raw_data"

# Load Silver Data
df_silver = spark.read.table(f"{catalog}.{schema}.silver_drug_reviews")

print(f"Loaded Silver Data. Total Records: {df_silver.count()}")

#### 2. NUMERICAL VARIABLE ANALYSIS

In [0]:
# CHECK FOR MISSING VALUES
print("----- Missing Values Check -----")
df_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in df_silver.columns]).show()

### Interpretation: Missing Values
* There are **899 rows** where the `condition` is null.
* To maintain medical accuracy, these conditions will not be imputed(guessed).Since the AI model predicts based on the `review` text, these missing labels do not hinder the training process.

In [0]:
# CHECK FOR DUPLICATE VALUES
total_count = df_silver.count()
distinct_count = df_silver.distinct().count()
duplicate_count = total_count - distinct_count

print("----- Duplicate Values Check -----")
print(f"Total Rows:    {total_count}")
print(f"Distinct Rows: {distinct_count}")
print(f"Duplicates:    {duplicate_count}")
if duplicate_count > 0:
    print("Duplicates found!")
else:
    print("No duplicates found.")

In [0]:
# VALIDITY & CONSISTENCY (Range Checks)
print("\n--- Validity & Range Checks ---")
df_silver.select(
    min("rating").alias("Min Rating (Should be 1)"), 
    max("rating").alias("Max Rating (Should be 10)"),
    min("usefulCount").alias("Min Useful (Should be >=0)"),
    max("event_date").alias("Latest Date")
).show()

### Interpretation:
* All ratings fall strictly between **1 and 10**, and there are no future dates.
* The dataset is highly robust with **0 duplicates**, ensuring that the model won't memorize repeated examples.

In [0]:
# CHECKING TEXT LENGTH FOR NLP USAGE
short_reviews = df_silver.filter(length(col("clean_review")) < 5)
short_count = short_reviews.count()

print(f"----- Text Length Check for NLP -----")
print(f"Short/Empty Reviews (Noise): {short_count}")

if short_count > 0:
    print("View Short/Empty Reviews:")
    display(short_reviews.select("rating", "clean_review", "drugName"))
else:
    print("No short reviews found. Data is clean")

### Interpretation: Noise Detection
* There are **10 reviews** with less than 5 characters (e.g., `"-"`, `"hi"`, `"Ok"`).
* As these are "Low Information" rows and they contain no sentiment or medical signal, these are **DROPPED** in the final cleaning step to prevent them from confusing the TF-IDF vectorizer.

In [0]:
# SUMMARY STATISTICS (Distribution)
# Understanding the spread of the numeric data
print("\n----- Summary Statistics -----")
df_silver.select("rating", "usefulCount").summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

### Interpretation: 
* **Statistical Skew:** The mean rating is **6.99**, indicating a "Positivity Bias" (patients are more likely to share success stories).

#### 3. CATEGORICAL VARIABLE ANALYSIS

In [0]:
from pyspark.sql.functions import countDistinct, col

print("----- Categorical Variable Analysis -----")

In [0]:
# Cardinality (How many unique drugs/conditions?)
# High cardinality in 'condition' suggests the data is covering many rare diseases.
df_silver.select(
    countDistinct("drugName").alias("Unique Drugs"),
    countDistinct("condition").alias("Unique Conditions")
).show()

With **3,436 unique drugs** and **884 conditions**, the data has a "long-tail" distribution. The model will likely perform better on common drugs than rare ones.

In [0]:
# Top 10 Conditions
print("Top 10 Conditions by Review Count:")
top_conditions = df_silver.groupBy("condition").count().orderBy(col("count").desc())
display(top_conditions.limit(10))

### Interpretation:
* The dataset is heavily skewed towards **Women's Health** (Birth Control: ~28,000 reviews) and **Mental Health** (Depression: ~9,000 reviews).
* The model will likely be biased toward these therapeutic areas. It may be less effective at detecting side effects for rare conditions (e.g., Oncology) due to lack of training data.

In [0]:
# Top 10 Drugs
print("Top 10 Drugs by Review Count:")
top_drugs = df_silver.groupBy("drugName").count().orderBy(col("count").desc())
display(top_drugs.limit(10))

#### 4. VISUALIZATION

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [0]:
volume_path = f"/Volumes/{catalog}/{schema}/landing_zone/Visualization_Plots.png"
sns.set_context("talk") 
sns.set_style("whitegrid") 

fig, axes = plt.subplots(1, 2, figsize=(22, 9))

# --- PLOT 1: RATINGS DISTRIBUTION (Left) ---
# Aggregating in Spark first
ratings_plot = df_silver.groupBy("rating").count().orderBy("rating").toPandas()

# Plot on the first axis (axes[0])
sns.barplot(
    x="rating", 
    y="count", 
    data=ratings_plot, 
    palette="viridis", 
    hue="rating", 
    legend=False,
    ax=axes[0]
)

axes[0].set_title("Distribution of Patient Ratings", fontsize=22, fontweight='bold', pad=20)
axes[0].set_xlabel("Satisfaction Score (1-10)", fontsize=16)
axes[0].set_ylabel("Count of Reviews", fontsize=16)
sns.despine(left=True, ax=axes[0]) # Remove left border line for modern look

# Add clean labels on top
for container in axes[0].containers:
    axes[0].bar_label(container, fontsize=12, padding=5, fmt='%.0f')


# --- PLOT 2: TOP 10 CONDITIONS (Right) ---
# Aggregating in Spark first
conditions_plot = (df_silver.groupBy("condition")
                  .count()
                  .orderBy(col("count").desc())
                  .limit(10)
                  .toPandas())

# Plot on the second axis (axes[1])
sns.barplot(
    x="count", 
    y="condition", 
    data=conditions_plot, 
    palette="rocket", 
    hue="condition", 
    legend=False,
    ax=axes[1]
)

axes[1].set_title("Top 10 Conditions (Domain Bias)", fontsize=22, fontweight='bold', pad=20)
axes[1].set_xlabel("Number of Reviews", fontsize=16)
axes[1].set_ylabel("") # Hide y-label as text explains it
sns.despine(left=True, ax=axes[1])

# Add clean labels to the right of the bars
for container in axes[1].containers:
    axes[1].bar_label(container, fontsize=12, padding=5, fmt='%.0f')


plt.tight_layout() 
plt.savefig(volume_path, dpi=300)
plt.show()
print(f"Saved to Volume: {volume_path}")

### Interpretation:
* **Ratings Distribution (Left Chart):** The data exhibits a clear **U-Shaped Distribution** (Polarity). The vast majority of reviews are either extremely negative (1/10) or extremely positive (10/10). This polarity is beneficial for a Binary Classification model as the signal between "Safe" and "Adverse" is distinct.
* **Condition Bias (Right Chart):** The dominance of "Birth Control" is visually confirmed. This suggests that **Class Imbalance** is a greater challenge in this project than Label Imbalance.

### Interpretation:
* **Ratings Distribution (Left Chart):** The data exhibits a clear **U-Shaped Distribution** (Polarity). The vast majority of reviews are either extremely negative (1-2) or extremely positive (9-10).
    * **Action:** This polarity is leveraged by **dropping the ambiguous "Neutral" ratings (5-6)** in the modeling stage to sharpen the decision boundary.
* **Condition Bias (Right Chart):** The visual dominance of "Birth Control" confirms significant **Domain Bias**.
    * **Action:** We will not downsample this (as it reflects real-world prevalence), but we will rely on **TF-IDF Vectorization** to penalize generic high-frequency terms associated with this dominant group, ensuring the model remains sensitive to rare signals in other conditions.

#### 5. FINAL CLEANING & SAVING

In [0]:

from pyspark.sql.functions import length, col

# 1. DROP the 10 "Noise" reviews (< 5 chars)
# 2. KEEP the 899 null conditions (Scientific Integrity)
df_cleaned = df_silver.filter(length(col("clean_review")) >= 5)

dropped_count = df_silver.count() - df_cleaned.count()
print(f"Dropped {dropped_count} noisy rows.")
print(f"Final Cleaned Count: {df_cleaned.count()}")

# Save to Silver (Cleaned)
table_name = f"{catalog}.{schema}.silver_drug_reviews_cleaned"
print(f"Saving to {table_name}...")

(df_cleaned.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(table_name)
)

print("Data is clean. Ready for modeling")